In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('imdb_movies_india.csv', encoding='ISO-8859-1')

In [3]:
df.drop(columns=['Name'], inplace=True) #not really useful

In [4]:
df['Year'] = df['Year'].astype(str).apply(lambda year: year.replace('(', '').replace(')', '')) #remove brackets around year
df.Duration = df.Duration.str.replace(' min', '') #remove ` min` to retain only numbers

In [5]:
df = df.applymap(lambda val: np.nan if val == 'nan' else val) #replace the string `nan` with the `NaN` type

In [6]:
#Format the votes to numbers
def format_vote(vote):
    if vote == 'nan':
        return np.nan
    vote = vote.replace('$', '')
    if "M" in vote:
        vote = vote.replace('M', '')
        f, s = vote.split('.')
        f, s = int(f), int(s)
        return ((f * 1000000) + (s * 10000))
    elif vote.count(',') == 1:
        return vote.replace(',', '')
    return vote

In [7]:
df.Year = pd.to_numeric(df.Year)
df.Duration = pd.to_numeric(df.Duration)
df.Votes =df.Votes.astype(str).apply(format_vote)
df.Votes = pd.to_numeric(df.Votes)

df.Rating = df.groupby('Genre')['Rating'].transform(lambda x: x.fillna(x.median()))
df.Votes = df.groupby('Genre')['Votes'].transform(lambda x: x.fillna(x.median()))
df.Rating.fillna(df.Rating.median(), inplace=True)
df.Votes.fillna(df.Votes.median(), inplace=True)

df.Duration.fillna(df.Duration.median(), inplace=True)
df.dropna(subset=['Year', 'Director'], inplace=True)

In [8]:
lists = []
for genre_list in df["Genre"]:
    lists.append(genre_list)
    
genres = []
for genre in lists:
    list_ = str(genre).split(',')
    for individual_genre in list_:
        genres.append(individual_genre.strip())
    
# Get all unique genres
all_genres = set(genres)

# Create binary columns for each genre
for genre in all_genres:
    df[genre] = df.Genre.apply(lambda genress: 1 if genre in str(genress) else 0)

# Drop the original Genre column and the nan after encoding
df.drop(columns=["Genre", 'nan'], inplace=True)
df

,Year,Duration,Rating,Votes,Director,Actor 1,Actor 2,Actor 3,Sci-Fi,Mystery,...,Biography,Reality-TV,Drama,Horror,Animation,Fantasy,Comedy,Action,Musical,History
1,2019.0,109.0,7.0,8.0,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid,0,0,...,0,0,1,0,0,0,0,0,0,0
2,2021.0,90.0,6.7,110.0,Soumyajit Majumdar,Sayani Gupta,Plabita Borthakur,Roy Angana,0,0,...,0,0,1,0,0,0,0,0,1,0
3,2019.0,110.0,4.4,35.0,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor,0,0,...,0,0,0,0,0,0,1,0,0,0
4,2010.0,105.0,6.5,27.0,Amol Palekar,Rajat Kapoor,Rituparna Sengupta,Antara Mali,0,0,...,0,0,1,0,0,0,0,0,0,0
5,1997.0,147.0,4.7,827.0,Rahul Rawail,Bobby Deol,Aishwarya Rai Bachchan,Shammi Kapoor,0,0,...,0,0,1,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15503,1989.0,125.0,5.8,44.0,S.P. Muthuraman,Chiranjeevi,Jayamalini,Rajinikanth,0,0,...,0,0,1,0,0,0,0,1,0,0
15504,1988.0,131.0,4.6,11.0,Mahendra Shah,Naseeruddin Shah,Sumeet Saigal,Suparna Anand,0,0,...,0,0,0,0,0,0,0,1,0,0
15505,1999.0,129.0,4.5,655.0,Kuku Kohli,Akshay Kumar,Twinkle Khanna,Aruna Irani,0,0,...,0,0,1,0,0,0,0,1,0,0
15506,2005.0,131.0,5.1,16.0,Kiran Thej,Sangeeta Tiwari,NaN,NaN,0,0,...,0,0,0,0,0,0,0,1,0,0


In [9]:
df[['Actor 1', 'Actor 2', 'Actor 3']] = df[['Actor 1', 'Actor 2', 'Actor 3']].fillna("Unknown")

In [10]:
import category_encoders as ce

# Define the columns that need encoding
actor_cols = ['Actor 1', 'Actor 2', 'Actor 3']

# Initialize TargetEncoder
encoder = ce.TargetEncoder(cols=actor_cols, smoothing=0.2) 

# Apply Target Encoding (using 'Rating' as the target)
df[actor_cols] = encoder.fit_transform(df[actor_cols], df['Rating'])

In [11]:
df.drop(columns=['Director'], inplace=True) 

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
model = RandomForestRegressor()

In [13]:
X = df.drop(columns=['Rating'])
y = df.Rating

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [15]:
model.fit(X_train, y_train)

RandomForestRegressor()

In [16]:
pred = model.predict(X_test)

In [17]:
mean_absolute_error(y_true=y_test, y_pred=pred)

0.4887606477665215